To-Do: entire workflow
chemdraw parse -> combinatorial expansion -> conformer generation -> aso description -> post-processing/dimension-reduction/analysis -> plot/output data

1. ChemDraw Parsing

In [3]:
import molli as ml
import molli.visual
import subprocess

In [4]:
cores = ml.files.box_cores_test_1
subs = ml.files.box_substituents_test_1

In [5]:
subprocess.run(['molli', 'parse', '--hadd', f'{cores}', '-o', '../ncsa-testing/ncsa-testing-data/BOX_cores_new_env.mlib'])
subprocess.run(['molli', 'parse', '--hadd', f'{subs}', '-o', '../ncsa-testing/ncsa-testing-data/BOX_subs_new_env.mlib'])

100%|██████████| 15/15 [00:00<00:00, 368.90it/s]


CompletedProcess(args=['molli', 'parse', '--hadd', '/home/ethangm2/NCSA Development/molli/molli/files/BOX_substituents_test_1.cdxml', '-o', '../ncsa-testing/ncsa-testing-data/BOX_subs_new_env.mlib'], returncode=0)

In [6]:
m_core = ml.MoleculeLibrary('ncsa-testing-data/BOX_cores_new_env.mlib')
print(len(m_core))
m_core[0]

5


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Molecule(name='1', formula='C10 H12 Br2 N2 O2 Unknown2')

In [7]:
m_subs = ml.MoleculeLibrary('ncsa-testing-data/BOX_subs_new_env.mlib')
print(len(m_subs))
m_subs[0]

15


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Molecule(name='3', formula='C3 H7 Unknown1')

2. Combinatorial Expansion

In [8]:
subprocess.run(
    [
        'molli',
        'combine',
        'ncsa-testing-data/BOX_cores_new_env.mlib',
        '-s',
        'ncsa-testing-data/BOX_subs_new_env.mlib',
        '-j',
        '96', 
        '-o', 
        'ncsa-testing-data/test_combine_new_env.mlib', 
        '-a', 
        'A1', 
        '--obopt', 
        'uff',
        '-m',
        'same'
    ]
)


Will create a library of size 75


100%|██████████| 75/75 [00:01<00:00, 38.75it/s]


CompletedProcess(args=['molli', 'combine', 'ncsa-testing-data/BOX_cores_new_env.mlib', '-s', 'ncsa-testing-data/BOX_subs_new_env.mlib', '-j', '96', '-o', 'ncsa-testing-data/test_combine_new_env.mlib', '-a', 'A1', '--obopt', 'uff', '-m', 'same'], returncode=0)

In [9]:
combined = ml.MoleculeLibrary('ncsa-testing-data/test_combine_new_env.mlib')
print(len(combined))
combined[0]

75


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Molecule(name='1_3_3', formula='C16 H26 Br2 N2 O2')

In [10]:
combined[42]

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Molecule(name='3_6_6', formula='C30 H38 Br2 N2 O2')

3. Conformer Generation

In [11]:
subprocess.run(['molli', 
                'conformers', 
                'ncsa-testing-data/test_combine_new_env.mlib', 
                '-n', 
                '50', 
                '-o', 
                'ncsa-testing-data/test_conformers_new_env.mlib', 
                '-t', 
                '-p', 
                '100'])

100%|██████████| 75/75 [02:12<00:00,  1.77s/it]


CompletedProcess(args=['molli', 'conformers', 'ncsa-testing-data/test_combine_new_env.mlib', '-n', '50', '-o', 'ncsa-testing-data/test_conformers_new_env.mlib', '-t', '-p', '100'], returncode=0)

In [12]:
clib = ml.ConformerLibrary('ncsa-testing-data/test_conformers_new_env.mlib')
print(len(clib))

75


In [13]:
i = 0
for conf in clib:
    i += conf.n_conformers
    # print(conf)
print(str(i) + ' conformers in library')

8028 conformers in library


In [14]:
clib[0][0]

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Conformer(name='1_3_3', formula='C16 H26 Br2 N2 O2')

In [15]:
clib[0][1]

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Conformer(name='1_3_3', formula='C16 H26 Br2 N2 O2')

In [16]:
clib[42][0]

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Conformer(name='3_6_6', formula='C30 H38 Br2 N2 O2')

In [17]:
clib[42][24]

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Conformer(name='3_6_6', formula='C30 H38 Br2 N2 O2')

3.5. Create grid

In [18]:
subprocess.run(['molli', 
                'grid', 
                '--mlib', 
                'ncsa-testing-data/test_conformers_new_env.mlib', 
                '-o', 
                'ncsa-testing-data/grid_new_env.npy'
                ])

(7106, 3)


100%|██████████| 75/75 [00:00<00:00, 1659.65it/s]


CompletedProcess(args=['molli', 'grid', '--mlib', 'ncsa-testing-data/test_conformers_new_env.mlib', '-o', 'ncsa-testing-data/grid_new_env.npy'], returncode=0)

4. ASO description of molecules

In [19]:
subprocess.run(['molli', 
                'gbca', 
                'aso', 
                'ncsa-testing-data/test_conformers_new_env.mlib', 
                '-g', 
                'ncsa-testing-data/grid_new_env.npy', 
                '-o', 
                'ncsa-testing-data/aso_new_env.h5'
                ])
# tqdm looks messed up

Allocating storage for descriptors
Will compute descriptor ASO using 128 cores.
Grid shape: (7106, 3)


Loading batches of conformers: 0it [00:00, ?it/s]
Submitting calculations:   0%|          | 0/75 [00:00<?, ?it/s]
                                                               

Gathering calculation results:   0%|          | 0/75 [00:00<?, ?it/s]

Loading batches of conformers: 1it [00:13, 13.52s/it]                


CompletedProcess(args=['molli', 'gbca', 'aso', 'ncsa-testing-data/test_conformers_new_env.mlib', '-g', 'ncsa-testing-data/grid_new_env.npy', '-o', 'ncsa-testing-data/aso_new_env.h5'], returncode=0)

In [20]:
# subprocess.run(['molli', 'grid', '--mlib', '../../out_conformers1/test_conformers.mlib', '-o', 'molli/lib_gen/test_aso'])


In [21]:
#subprocess.run(['molli', 'descriptors', '../../out_conformers1/test4_conformers.mlib', '-o', '../../test_aso_data/test_aso.h5'])

5. post-processing/dimension-reduction/analysis

In [22]:
subprocess.run(         # check functionality for plotting and pca
     [                   # should be better way to implement post_processing stuff
         'molli', 
         'cluster', 
         'ncsa-testing-data/aso_new_env.h5', 
         '-o', 
         '/scratch/ethangm2/cluster-test/new_env_data3', 
         '-v',
         '0',
         '-c',
         '0.8',
     ]
 )

100%|██████████| 75/75 [00:00<00:00, 4697.99it/s]


shape of data after variance threshold: (75, 3784)
total variance after variance threshold: 62.94

shape of data after removing correlated columns (R > 0.8): (75, 254)
total variance after removing correlated columns (R > 0.8): 3.39



CompletedProcess(args=['molli', 'cluster', 'ncsa-testing-data/aso_new_env.h5', '-o', '/scratch/ethangm2/cluster-test/new_env_data3', '-v', '0', '-c', '0.8'], returncode=0)